In [ ]:
import requests
import json
from secrets import creds

DEBUG = False
headers = {
    'Content-Type': 'application/json-rpc',
}

In [ ]:
def send_request(headers, request):
    response = requests.post(creds.API_URL, headers=headers, data=json.dumps(request).encode('utf-8'))
    return response.json()

In [ ]:
# Setup Authentication
request = {
    'jsonrpc': '2.0',
    'method': 'user.login',
    'params': {
        'user': creds.API_USER,
        'password': creds.API_PASS
    },
    'id': 1,
}
auth = send_request(headers, request)

if 'result' not in auth:
    print('Authentication Failed')
    exit()

api_key = auth['result']
    

In [ ]:
# Extract all monitored hosts
request = {
    'jsonrpc': '2.0',
    'method': 'host.get',
    'params': {
        'monitored_hosts': 'true'
    },
    'auth': api_key,
    'id': 1
}
hosts = send_request(headers, request)

if 'result' not in hosts:
    print('Failed to get hosts')
    exit()

if DEBUG:
    print(json.dumps(hosts['result'], indent=4))

In [ ]:
# Convert result of monitored hosts into a list of host IDs
if len(hosts['result']) == 0:
    print('No monitored hosts found')
    exit()

host_ids = [host['hostid'] for host in hosts['result']]

if DEBUG:
    print(host_ids)

In [ ]:
# Get Active Triggers
request = {
    'jsonrpc': '2.0',
    'method': 'trigger.get',
    'params': {
        'filter': {
            'value': 1
        },
        'sortfield': 'lastchange',
        'sortorder': 'DESC',
        'monitored': 'true',
        'active': 'true',
        'hostids': host_ids,
        'only_true': 'true',
        'expandDescription': 'true',
        "output": "extend",
        "selectHosts": "extend",
    },
    'auth': api_key,
    'id': 1
}
triggers = send_request(headers, request)

if DEBUG:
    print(json.dumps(triggers, indent=4))

In [ ]:
# Handle Triggers
trigger_severity = {
    0: 'Not classified',
    1: 'Information',
    2: 'Warning',
    3: 'Average',
    4: 'High',
    5: 'Disaster'
}

for i in range(len(trigger_severity)):
    severity_triggers = []

    for trigger in triggers['result']:
        if trigger['priority'] == str(i):
            severity_triggers += [trigger]

    print("\n%s (%i)" % (
        trigger_severity[i], 
        len(severity_triggers)))

    for t in severity_triggers:
        print("%s: %s" % (
            t['hosts'][0]['host'].split('.')[0], 
            t['description']))